<a href="https://colab.research.google.com/github/timbocf/image-to-video-generator/blob/main/ComfyUI_colab_i2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

===== Colab bootstrap for fast ComfyUI startup (GPU-ready + updates + Drive caching) =====

In [1]:
# ===============================
# ComfyUI One-Cell Colab Setup
# ===============================

# 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, subprocess, shutil, sys, time, pathlib

# -------------------------------
# Paths (persisted in Drive)
# -------------------------------
DRIVE_ROOT = '/content/drive/MyDrive/ComfyUI'
MODELS_DRIVE = os.path.join(DRIVE_ROOT, 'models')
NODES_DRIVE = os.path.join(DRIVE_ROOT, 'custom_nodes')
CACHE_DRIVE = os.path.join(DRIVE_ROOT, 'pipcache')
NGROK_TOKEN_FILE = os.path.join(DRIVE_ROOT, 'ngrok_token.txt')
COMFY_PASS_FILE = os.path.join(DRIVE_ROOT, 'comfy_password.txt')

os.makedirs(MODELS_DRIVE, exist_ok=True)
os.makedirs(NODES_DRIVE, exist_ok=True)
os.makedirs(CACHE_DRIVE, exist_ok=True)

# -------------------------------
# 2) Pip cache to Drive
# -------------------------------
os.environ['PIP_CACHE_DIR'] = CACHE_DRIVE

# -------------------------------
# 3) Clone / Update ComfyUI
# -------------------------------
REPO_DIR = '/content/ComfyUI'
COMFYUI_GIT = 'https://github.com/comfyanonymous/ComfyUI.git'

if not os.path.exists(REPO_DIR):
    subprocess.run(['git', 'clone', COMFYUI_GIT, REPO_DIR], check=True)
else:
    try:
        subprocess.run(['git', '-C', REPO_DIR, 'fetch'], check=True)
        subprocess.run(['git', '-C', REPO_DIR, 'reset', '--hard', 'origin/main'], check=True)
    except Exception as e:
        print("Warning: git update failed:", e)

# -------------------------------
# 4) Symlink models & nodes
# -------------------------------
REPO_MODELS = os.path.join(REPO_DIR, 'models')
REPO_CUSTOM_NODES = os.path.join(REPO_DIR, 'custom_nodes')

def safe_link(src, dst):
    if os.path.islink(dst) or os.path.exists(dst):
        try:
            if os.path.islink(dst):
                os.unlink(dst)
            elif os.path.isdir(dst):
                shutil.rmtree(dst)
            else:
                os.remove(dst)
        except Exception as e:
            print("Warning removing:", dst, e)
    os.symlink(src, dst)

safe_link(MODELS_DRIVE, REPO_MODELS)
safe_link(NODES_DRIVE, REPO_CUSTOM_NODES)

# -------------------------------
# 5) Virtual Environment
# -------------------------------
VENV_DIR = '/content/venv_comfy'
if not os.path.exists(VENV_DIR):
    subprocess.run([sys.executable, '-m', 'venv', VENV_DIR], check=True)

bin_py = os.path.join(VENV_DIR, 'bin', 'python')
pip_cmd = os.path.join(VENV_DIR, 'bin', 'pip')

subprocess.run([pip_cmd, 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True)

# -------------------------------
# 6) Torch install (GPU-safe)
# -------------------------------
def has_gpu():
    try:
        subprocess.check_output(['nvidia-smi'], stderr=subprocess.DEVNULL)
        return True
    except Exception:
        return False

if has_gpu():
    print("GPU detected — installing CUDA torch")
    try:
        subprocess.run([
            pip_cmd, 'install',
            'torch', 'torchvision', 'torchaudio',
            '--index-url', 'https://download.pytorch.org/whl/cu118'
        ], check=True)
    except Exception:
        print("CUDA install failed, falling back to CPU torch")
        subprocess.run([
            pip_cmd, 'install',
            'torch',
            '--index-url', 'https://download.pytorch.org/whl/cpu'
        ], check=True)
else:
    print("No GPU detected — CPU torch")
    subprocess.run([
        pip_cmd, 'install',
        'torch',
        '--index-url', 'https://download.pytorch.org/whl/cpu'
    ], check=True)

# -------------------------------
# 7) ComfyUI requirements
# -------------------------------
req_file = os.path.join(REPO_DIR, 'requirements.txt')
if os.path.exists(req_file):
    subprocess.run([pip_cmd, 'install', '-r', req_file], check=False)

# -------------------------------
# 8) Optional


SyntaxError: invalid syntax (ipython-input-1473436082.py, line 5)